In [162]:
import pandas as pd
import warnings
import requests
from bs4 import BeautifulSoup
import re
warnings.filterwarnings('ignore')

# Import merged csv of raw data. Data courtesy of baseball-reference.com
df = pd.read_csv (r'/Users/lochlanhendrix/Desktop/TripleAHitters.csv')

#Add column for what team a player is on
team = []
for i in range (0, 39):
    team.append("El Paso")
for i in range (39, 62):
    team.append("Oklahoma City")
for i in range (62,98):
    team.append("Round Rock")
for i in range (98,127):
    team.append("Sugar Land")
for i in range (127, 155):
    team.append("Albuquerque")
for i in range (155, 192):
    team.append("Reno")
for i in range (192, 237):
    team.append("Tacoma")
for i in range (237, 279):
    team.append("Las Vegas")
for i in range (279, 315):
    team.append("Salt Lake")
for i in range (315, 362):
    team.append("Sacremento")   
for i in range (362, 396):
    team.append("Durham")
for i in range (396,425):
    team.append("Scranton")
for i in range (425,465):
    team.append("Jacksonville")
for i in range (465, 501):
    team.append("Buffalo")
for i in range (501, 542):
    team.append("Lehigh Valley")    
for i in range (542, 573):
    team.append("Worcester") 
for i in range (573, 614):
    team.append("Norfolk")
for i in range (614, 649):
    team.append("Rochester")  
for i in range (649, 689):
    team.append("Syracuse") 
for i in range (689, 724):
    team.append("Charlotte")  
for i in range (724, 760):
    team.append("Nashville")  
for i in range (760, 793):
    team.append ("Toledo")   
for i in range (793, 831):
    team.append("Columbus")   
for i in range (831, 871):
    team.append("Indianapolis")  
for i in range (871, 915):
    team.append("St. Paul")   
for i in range (915, 946):
    team.append("Memphis")  
for i in range (946, 978):
    team.append("Omaha")   
for i in range (978, 1011):
    team.append("Gwinnett") 
for i in range (1011, 1044):
    team.append("Iowa")
for i in range (1044, 1091):
    team.append("Louisville")    
    
df['Team'] = team


#Add column for what league a player is in
league = []
for i in range (0, 362 ):
    league.append("Pacific Coast League")
for i in range (362, 1091):
    league.append("International League")
df['League'] = league



Name = []
Name = df["Name"]
Bats = []

#Raw data is formatted so that a player that bats left will be denoted as FirstName LastName*, and a switch will be
#FirstName LastName#. Remove symbols from end of name and instead add a column that denotes batting preference.
for i in range (0, len(Name) ):
    
    iLen = len(Name[i])
    
    if (Name[i][iLen - 1] == '*'):
        newName = Name[i].replace('*','')
        Name[i] = newName
        Bats.append("Left")
        
    elif (Name[i][iLen - 1] == '#'):
        newName = Name[i].replace('#','')
        Name[i] = newName
        Bats.append("Switch")
        
    else:
        Bats.append("Right")
        
df['Name'] = Name
df['Bats'] = Bats

#Remove junk columns
df.drop('Rk', inplace = True, axis = 1)
df.drop('Name-additional', inplace = True, axis = 1)
df.drop('Notes', inplace = True, axis = 1)

#Export formatted data
#Dataset represents players that are currently in the minor leagues whose future major league success will be predicted
df.to_csv (r'/Users/lochlanhendrix/Desktop/TripleA_DF.csv', index = None, header=True) 

#Also want major league data, pull from ESPN
#I suck at this, credit to https://www.codecademy.com/resources/blog/web-scraping-python-beautiful-soup-mlb-stats/

url = 'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2021/start/1'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
header = soup.find('tr', attrs = {'class': 'colhead'})
columns = [col.get_text() for col in header.find_all('td')]
newDF = pd.DataFrame(columns = columns)

for i in range(1,300,50):
    url = 'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2021/start/{}'.format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    players = soup.find_all('tr', attrs = {'class':re.compile('row player-10-')})
    
    for player in players:
        stats = [stat.get_text() for stat in player.find_all('td')]
        tempDF = pd.DataFrame(stats).transpose()
        tempDF.columns = columns
        
        newDF = pd.concat([newDF,tempDF], ignore_index = True)

#Convert columns from string object to float

for i in range(2,len(columns)):
    colToChange = columns[i]
    newDF[colToChange] = pd.to_numeric(newDF[colToChange])
    
#Add column for OBP

OBP = []

for i in range (0,newDF.shape[0]):
    OBPi = (newDF.iloc[i,6] + newDF.iloc[i,11]) / (newDF.iloc[i,4] + newDF.iloc[i,11])
    OBP.append(OBPi)
    
newDF['OBP'] = OBP
    
    
    
#Export Major League dataset

newDF.to_csv (r'/Users/lochlanhendrix/Desktop/MajorLeagueDF.csv', index = None, header=True)




,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA,OBP
0,1,Luis Robert,2,68,275,42,93,22,1,13,43,14,61,6,1,0.338,0.370242
1,2,Trea Turner,7,148,595,107,195,34,3,28,77,41,110,32,5,0.328,0.371069
2,3,Frank Schwindel,2,64,242,44,79,20,1,14,43,16,41,2,1,0.326,0.368217
3,4,Yuli Gurriel,6,143,530,83,169,31,0,15,81,59,68,1,1,0.319,0.387097
4,5,Ketel Marte,7,90,340,52,108,29,1,14,50,31,60,2,0,0.318,0.374663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,,Jon Berti,4,85,233,35,49,10,1,4,19,32,61,8,4,0.210,0.305660
296,,David Dahl,5,63,205,19,43,11,0,4,18,10,59,2,1,0.210,0.246512
297,,Matt Chapman,5,151,529,75,111,15,3,27,72,80,202,3,2,0.210,0.313629
298,,Jake Fraley,3,78,214,27,45,7,0,9,36,46,71,10,2,0.210,0.350000
